In [2]:
import numpy as np
import pandas as pd

In [3]:
match = pd.read_csv('matches.csv')
delivery = pd.read_csv('deliveries.csv')

In [4]:
total_score_df = delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()

C:\Users\soumi\AppData\Local\Temp\ipykernel_6116\2553512226.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_score_df = delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()


In [5]:
total_score_df = total_score_df[total_score_df['inning'] == 1]

In [6]:
match_df = match.merge(total_score_df[['match_id','total_runs']], left_on='id', right_on='match_id')

In [7]:
teams = [
    'Sunrisers Hyderabad',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Kolkata Knight Riders',
    'Kings XI Punjab',
    'Chennai Super Kings',
    'Rajasthan Royals',
    'Delhi Capitals'
]

In [8]:
match_df['team1'] = match_df['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['team2'] = match_df['team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['team1'] = match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2'] = match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [9]:
match_df = match_df[match_df['team1'].isin(teams)]
match_df = match_df[match_df['team2'].isin(teams)]

In [10]:
match_df = match_df[match_df['dl_applied'] == 0]

In [11]:
match_df = match_df[['match_id','city','winner','total_runs']]

In [12]:
delivery_df = match_df.merge(delivery, on='match_id')

In [13]:
delivery_df = delivery_df[delivery_df['inning'] == 2]

In [14]:
#Making an extra column in delivery_df named current_score
delivery_df['current_score'] = delivery_df.groupby('match_id').cumsum()['total_runs_y']

C:\Users\soumi\AppData\Local\Temp\ipykernel_6116\4218244993.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  delivery_df['current_score'] = delivery_df.groupby('match_id').cumsum()['total_runs_y']


In [15]:
delivery_df['runs_left'] = (delivery_df['total_runs_x']  - delivery_df['current_score']) + 1
#One more column added namely runs_left

In [16]:
delivery_df['balls_left'] = 126 - (delivery_df['over']*6 + delivery_df['ball'])

In [17]:
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].fillna("0") #Replacing all NaN values to 0
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].apply(lambda x:x if x == "0" else "1") #Replacing all player names to 1
delivery_df['player_dismissed'] = delivery_df['player_dismissed'].astype('int')#Coverting all values to integers
wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values #Cummulative Sum of the column to get the number of wickets


C:\Users\soumi\AppData\Local\Temp\ipykernel_6116\2149457274.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values #Cummulative Sum of the column to get the number of wickets


In [18]:
delivery_df['wickets_left'] = 10 - wickets # Substarcting from 10 to get wickets_left and creating new col in delivery_df

In [19]:
delivery_df['crr'] = (delivery_df['current_score']*6) / (120 - delivery_df['balls_left'])

In [20]:
delivery_df['rrr'] = (delivery_df['runs_left']*6) / delivery_df['balls_left']

In [21]:
def result(row):
    return 1 if row['batting_team'] == row['winner'] else 0

In [22]:
delivery_df['result'] = delivery_df.apply(result,axis=1)

In [23]:
final_df = delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets_left','total_runs_x','crr','rrr','result']]

In [24]:
final_df = final_df.sample(final_df.shape[0])

In [25]:
final_df.isnull().sum()
#finding the presence of any NaN values which could create future problems. 

batting_team      0
bowling_team      0
city            832
runs_left         0
balls_left        0
wickets_left      0
total_runs_x      0
crr               0
rrr               5
result            0
dtype: int64

In [26]:
final_df.dropna(inplace=True) #Dropping the NaN values

In [27]:
final_df = final_df[final_df['balls_left'] != 0] #rrr can be NaN when no of balls(denominator) is zero. So removing those rows

In [28]:
teams

['Sunrisers Hyderabad',
 'Mumbai Indians',
 'Royal Challengers Bangalore',
 'Kolkata Knight Riders',
 'Kings XI Punjab',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Delhi Capitals']

In [29]:
X = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [32]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])

In [33]:
pipe.fit(X_train,y_train)

C:\Users\soumi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['batting_team',
                                                   'bowling_team', 'city'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [34]:
y_pred = pipe.predict(X_test)

In [35]:
#Testing Accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8049618053122153

In [36]:
teams

['Sunrisers Hyderabad',
 'Mumbai Indians',
 'Royal Challengers Bangalore',
 'Kolkata Knight Riders',
 'Kings XI Punjab',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Delhi Capitals']

In [37]:
delivery_df['city'].unique()

array(['Hyderabad', 'Bangalore', 'Mumbai', 'Indore', 'Kolkata', 'Delhi',
       'Chandigarh', 'Jaipur', 'Chennai', 'Cape Town', 'Port Elizabeth',
       'Durban', 'Centurion', 'East London', 'Johannesburg', 'Kimberley',
       'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi', 'Abu Dhabi',
       'Sharjah', nan, 'Mohali', 'Bengaluru'], dtype=object)

In [38]:
import pickle
pickle.dump(pipe, open('pipe.pkl','wb'))

In [40]:
final_df

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,total_runs_x,crr,rrr,result
121179,Kolkata Knight Riders,Sunrisers Hyderabad,Delhi,95,57,7,162,6.476190,10.000000,0
17023,Delhi Daredevils,Royal Challengers Bangalore,Bangalore,65,78,9,154,12.857143,5.000000,1
139802,Sunrisers Hyderabad,Mumbai Indians,Hyderabad,125,106,10,142,7.714286,7.075472,0
101874,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata,149,91,9,177,6.000000,9.824176,1
84251,Delhi Daredevils,Chennai Super Kings,Chennai,81,44,5,168,6.947368,11.045455,0
...,...,...,...,...,...,...,...,...,...,...
73459,Sunrisers Hyderabad,Royal Challengers Bangalore,Hyderabad,60,46,7,130,5.756757,7.826087,1
47750,Mumbai Indians,Chennai Super Kings,Mumbai,127,68,9,168,4.846154,11.205882,0
56211,Delhi Daredevils,Kings XI Punjab,Dharamsala,69,24,4,170,6.375000,17.250000,0
65155,Rajasthan Royals,Delhi Daredevils,Delhi,148,118,10,152,15.000000,7.525424,0
